### Zadanie 2: System rezerwacji lotów
**Zadanie**: Stwórz bazę faktów dla systemu rezerwacji lotów, który obejmuje lotniska, połączenia między lotniskami oraz ceny biletów. Zdefiniuj reguły `polaczenie(X, Y)`, `koszt_podrozy(X, Y, C)`, `najkrotsza_droga(X, Y)`.

**Fakty**
```prolog
lotnisko(świdnik).
lotnisko(kraków).
lotnisko(radom).
lotnisko(gdańsk).
lotnisko(warszawa).
%lot(początek, koniec, cena).
lot(krakow, radom, 70).
lot(krakow, gdansk, 400).
lot(radom, gdansk, 100).
lot(swidnik, radom, 50).
lot(swidnik, warszawa, 60).
lot(warszawa, gdansk, 200).
lot(radom, krakow, 50).
lot(gdansk, krakow, 360).
lot(gdansk, radom, 120).
lot(radom, swidnik, 40).
lot(warszawa, swidnik, 60).
lot(gdansk, warszawa, 150).
```
**Reguły**
```prolog
% polaczenie(X, Y) sprawdza, czy istnieje bezpośrednie połączenie lotnicze między lotniskiem X a lotniskiem Y.

polaczenie(X, Y) :-
    lot(X, Y, _), !. % _ ponieważ nie ważne jaka cena tam jest, połączenie istnieje. ! ponieważ wystarczy aby znalaziono jedno polaczenie.

% koszt_podrozy(X, Y, C) podaje koszt podróży bezpośredniej z X do Y.
koszt_podrozy(X, Y, C) :-
    lot(X, Y, C).

droga(X, Y, Koszt, Trasa) :-
    droga2(X, Y, [X], Koszt, Trasa).

droga2(X, Y, _, Koszt, [X, Y]) :-
    lot(X, Y, Koszt).

droga2(X, Y, Odwiedzone, KosztCalk, [X|ResztaTrasy]) :-
    lot(X, Z, KosztXZ),
    \+ member(Z, Odwiedzone),       
    Z \= Y, % dodałam warunek, że miasto przesiadki nie może być miastem docelowym, ponieważ pokazywało trasy typu: radom świdnik warszawa gdańsk warszawa.
    droga2(Z, Y, [Z|Odwiedzone], KosztZy, ResztaTrasy),
    KosztCalk is KosztXZ + KosztZy.

najtansza_droga(X, Y) :-
    findall(Koszt, droga(X, Y, Koszt, _), Drogi),  % tworzy liste kosztów dróg z X do Y
    min_list(Drogi, H),      % znajduje minimalny koszt                   
    droga(X, Y, H, T),           % dopisuje inne dane drogi do tego kosztu                
    write('Najtańsza droga z '), write(X), write(' do '), write(Y),
    write(' jest przez '), write(T), write(' i kosztuje '), write(H), !.    % dodałam ! ponieważ w SWI przy niektórch pytaniach najtansza_droga odpowiadało pisząc trzy razy to samo zdanie z najtańszą drogą. W SWISH zawsze odpowiadało jednym zdaniem.
```

**Pytania i odpowiedzi**:
```prolog
?- polaczenie(krakow, radom).
true

?- polaczenie(gdansk, swidnik).
false % nie ma bezpośredniego połączenia.

?- koszt_podrozy(radom, swidnik, C).
C = 40

?- koszt_podrozy(swidnik, radom, C).
C = 50

?- koszt_podrozy(swidnik, gdansk, C).
false % nie istnieje takie połączenie.

?- droga(gdansk, swidnik, Koszt, Trasa). % podało wszytskie 3 możliwe trasy
Koszt = 470,
Trasa = [gdansk, krakow, radom, swidnik] ;
Koszt = 160,
Trasa = [gdansk, radom, swidnik] ;
Koszt = 210,
Trasa = [gdansk, warszawa, swidnik] ;
false.

?- droga(gdansk, radom, Koszt, Trasa).
Koszt = 120,
Trasa = [gdansk, radom] ;
Koszt = 430,
Trasa = [gdansk, krakow, radom] ;
Koszt = 260,
Trasa = [gdansk, warszawa, swidnik, radom] ;
false.

?- najtansza_droga(warszawa, radom).
Najtańsza droga z warszawa do radom jest przez [warszawa, swidnik, radom] i kosztuje 110
true

?- najtansza_droga(gdansk, radom).
Najtańsza droga z gdansk do radom jest przez [gdansk, radom] i kosztuje 120
true

?- najtansza_droga(swidnik, radom).
Najtańsza droga z swidnik do radom jest przez [swidnik, radom] i kosztuje 50
true

?- najtansza_droga(krakow, warszawa).
Najtańsza droga z krakow do warszawa jest przez [krakow, radom, swidnik, warszawa] i kosztuje 170
true
```